# How to select gridcells using vectorized indexing

## What is vectorized indexing anyway?

## Packages

We'll use `xarray` and `pandas` in this how-to.  We will also use `numpy` to create some testdata.

In [1]:
import xarray as xr
import pandas as pd

import numpy as np

## Create some test data

We'll create a `DataArray` to demonstrate the selection method.  The test `DataArray` is a global grid with a 2 degree spacing for two dates.  The data are an integer array with the values from 0 to `ntime*nlat*nlon`.

In [2]:
time = pd.date_range('2023-08-01', '2023-08-10', freq='D')
latitude = np.linspace(-90., 90., 91)
longitude = np.linspace(-180., 179., 360)

ntime = len(time)
nlat = len(latitude)
nlon = len(longitude)

data = np.arange(0, ntime*nlat*nlon).reshape(ntime, nlat, nlon)  # 

da = xr.DataArray(data, coords=[time, latitude, longitude], dims=['time', 'latitude', 'longitude'])
da

<xarray.DataArray (time: 10, latitude: 91, longitude: 360)>
array([[[     0,      1,      2, ...,    357,    358,    359],
        [   360,    361,    362, ...,    717,    718,    719],
        [   720,    721,    722, ...,   1077,   1078,   1079],
        ...,
        [ 31680,  31681,  31682, ...,  32037,  32038,  32039],
        [ 32040,  32041,  32042, ...,  32397,  32398,  32399],
        [ 32400,  32401,  32402, ...,  32757,  32758,  32759]],

       [[ 32760,  32761,  32762, ...,  33117,  33118,  33119],
        [ 33120,  33121,  33122, ...,  33477,  33478,  33479],
        [ 33480,  33481,  33482, ...,  33837,  33838,  33839],
        ...,
        [ 64440,  64441,  64442, ...,  64797,  64798,  64799],
        [ 64800,  64801,  64802, ...,  65157,  65158,  65159],
        [ 65160,  65161,  65162, ...,  65517,  65518,  65519]],

       [[ 65520,  65521,  65522, ...,  65877,  65878,  65879],
        [ 65880,  65881,  65882, ...,  66237,  66238,  66239],
        [ 66240,  66241,  66242, ...,  66597,  66598,  66599],
        ...,
...
        ...,
        [261000, 261001, 261002, ..., 261357, 261358, 261359],
        [261360, 261361, 261362, ..., 261717, 261718, 261719],
        [261720, 261721, 261722, ..., 262077, 262078, 262079]],

       [[262080, 262081, 262082, ..., 262437, 262438, 262439],
        [262440, 262441, 262442, ..., 262797, 262798, 262799],
        [262800, 262801, 262802, ..., 263157, 263158, 263159],
        ...,
        [293760, 293761, 293762, ..., 294117, 294118, 294119],
        [294120, 294121, 294122, ..., 294477, 294478, 294479],
        [294480, 294481, 294482, ..., 294837, 294838, 294839]],

       [[294840, 294841, 294842, ..., 295197, 295198, 295199],
        [295200, 295201, 295202, ..., 295557, 295558, 295559],
        [295560, 295561, 295562, ..., 295917, 295918, 295919],
        ...,
        [326520, 326521, 326522, ..., 326877, 326878, 326879],
        [326880, 326881, 326882, ..., 327237, 327238, 327239],
        [327240, 327241, 327242, ..., 327597, 327598, 327599]]])
Coordinates:
  * time       (time) datetime64[ns] 2023-08-01 2023-08-02 ... 2023-08-10
  * latitude   (latitude) float64 -90.0 -88.0 -86.0 -84.0 ... 86.0 88.0 90.0
  * longitude  (longitude) float64 -180.0 -179.0 -178.0 ... 177.0 178.0 179.0

## Create some points to select

Here, I'm using coordinates of London, Paris, New York and Tokyo.

First, we'll use these coordinates just in `xarray` to introduce the method.  However, in most _real-world_ situations, the points will be in a file.  Something like a csv.  So we will also convert a `pandas.DataFrame` to an xarray object to demonstrate this workflow.

### Using `xarray` only

We use the `.sel()` indexing method.  `.sel()` allows us to specify a selection method as a keyword.  The methods are `pad` which fills forward, `backfill` which fills backward, and `nearest` which does a nearest-neighbour search.  If no method is selected, only exact matches are returned.  For geospatial queries, this will almost always return an empty set of results.To use vectorized subsetting, the points to select have to be `xarray.DataArrays`.  If 

In [3]:
# Create some points to select.  These have to be DataArray objects to do vectorized subsetting
ptime = xr.DataArray(['2023-08-09'], dims=['time'])
plat = xr.DataArray([51.5072, 48.8566, 40.7128, 35.6762], dims=['point'])  # Points to select need common dimensions
plon = xr.DataArray([0.1276, 2.3522, 74.0060, 139.6503], dims=['point'])

da.sel(time=ptime, latitude=plat, longitude=plon, method='nearest').squeeze().to_pandas()  # Squeeze to drop single time export as pandas series

point
0    287820
1    287102
2    285734
3    285080
dtype: int64

### Using points from a `pandas.DataFrame`

In [4]:
df = pd.DataFrame({
    'latitude': [51.5072, 48.8566, 40.7128, 35.6762],
    'longitude': [0.1276, 2.3522, 74.0060, 139.6503],
},
    index = [1, 2, 3, 4],
                 )
df.index.name = 'points'
df

,latitude,longitude
points,,
1,51.5072,0.1276
2,48.8566,2.3522
3,40.7128,74.0060
4,35.6762,139.6503


In [5]:
ds_pts = df.to_xarray()
ds_pts

<xarray.Dataset>
Dimensions:    (points: 4)
Coordinates:
  * points     (points) int64 1 2 3 4
Data variables:
    latitude   (points) float64 51.51 48.86 40.71 35.68
    longitude  (points) float64 0.1276 2.352 74.01 139.7

In [15]:
da.sel(time=ptime, latitude=ds_pts.latitude, longitude=ds_pts.longitude, method='nearest')

<xarray.DataArray (time: 1, points: 4)>
array([[287820, 287102, 285734, 285080]])
Coordinates:
  * time       (time) datetime64[ns] 2023-08-09
    latitude   (points) float64 52.0 48.0 40.0 36.0
    longitude  (points) float64 0.0 2.0 74.0 140.0
  * points     (points) int64 1 2 3 4